# Train VGG16 on Custom Dataset

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive/')
path = "/content/drive/MyDrive/ANN-Project/"

Mounted at /content/drive/


In [ ]:
# Imports
import torch
import torch.nn as nn  # All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
import torch.nn.functional as F
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torchvision.transforms as transforms  # Transformations we can perform on our dataset
import torchvision.models as models
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader,
)  # Gives easier dataset managment and creates mini batches
from PIL import Image

class Tekken3Dataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        PIL_image = Image.fromarray (image)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))

        if self.transform:
            image = self.transform(PIL_image)

        return (image, y_label)


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
learning_rate = 1e-3
batch_size = 32
num_epochs = 10

# Specify transforms using torchvision.transforms as transforms library
transformations = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Data
dataset = Tekken3Dataset(
    csv_file=path+"dataset_only_images.csv",
    root_dir=path+"dataset",
    transform=transformations,
)

# 70:30 -> 7808

train_set, test_set = torch.utils.data.random_split(dataset, [5465, 2343])
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=True)

# Model

model = models.vgg16()
model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

# Check accuracy on training to see how good our model is
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

        print(
            f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}"
        )

    model.train()


print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Cost at epoch 0 is 1.6913418173789978
Cost at epoch 1 is 0.7017916963811506
Cost at epoch 2 is 0.6994726696209601
Cost at epoch 3 is 0.6969407943257114
Cost at epoch 4 is 0.6965081405918501
Cost at epoch 5 is 0.6962810294669971
Cost at epoch 6 is 0.6965277198462458
Cost at epoch 7 is 0.694534480920312
Cost at epoch 8 is 0.6961131566449216
Cost at epoch 9 is 0.6957225611335353
Checking accuracy on Training Set
Got 2745 / 5465 with accuracy 50.23
Checking accuracy on Test Set
Got 1160 / 2343 with accuracy 49.51


Extra Epoches

In [ ]:
# Train Network
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
        # Get data to cuda if possible
        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

print("Checking accuracy on Training Set")
check_accuracy(train_loader, model)

print("Checking accuracy on Test Set")
check_accuracy(test_loader, model)

Cost at epoch 0 is 0.6953630823838083
Cost at epoch 1 is 0.6961498131528933
Cost at epoch 2 is 0.6946044417849758
Cost at epoch 3 is 0.69536895082708
Cost at epoch 4 is 0.6943775234166641
Cost at epoch 5 is 0.6939951366151286
Cost at epoch 6 is 0.6940497574750443
Cost at epoch 7 is 0.6957850968628599
Cost at epoch 8 is 0.6947413341343751
Cost at epoch 9 is 0.6943481114872715
Checking accuracy on Training Set
Got 2720 / 5465 with accuracy 49.77
Checking accuracy on Test Set
Got 1183 / 2343 with accuracy 50.49


# Export Model Data

In [ ]:
torch.save(model.state_dict(), path+'model_weights.pth')

In [ ]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

# Desired Deep Features

In [ ]:
temp = model.classifier[3].weight

In [ ]:
export_data = temp.cpu().detach().numpy()
export_data

array([[-0.01472528, -0.0082471 ,  0.00089048, ..., -0.01633802,
         0.00478626, -0.00193277],
       [ 0.00957653, -0.00540264,  0.00794996, ..., -0.0094647 ,
        -0.01219382, -0.00268113],
       [-0.00880944, -0.00980182, -0.02101649, ..., -0.01422585,
        -0.00270699, -0.01039774],
       ...,
       [ 0.03772741, -0.01078148,  0.03850559, ..., -0.01457085,
         0.02113776, -0.00673066],
       [ 0.01711877, -0.01976011,  0.01861127, ..., -0.00514485,
         0.00497056, -0.0093764 ],
       [ 0.01096603, -0.0172389 ,  0.01574892, ...,  0.01127687,
         0.0073191 , -0.01291022]], dtype=float32)

In [ ]:
torch.save(model.classifier[3].weight, path+'layer_3_output_weights_tensor.pth')
torch.save(model.classifier[3].bias, path+'layer_3_output_bias_tensor.pth')

In [ ]:
torch.save(export_data, path+'layer_3_output_weights_numpy.pth')